In [11]:
import pandas as pd

df = pd.read_csv("customer_support_tickets.csv.zip")
df = df.dropna(subset=["Ticket Description", "Ticket Type", "Resolution"])


In [12]:
texts = df["Ticket Description"].values
labels = df["Ticket Type"].values
responses = df["Resolution"].values


In [13]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

le = LabelEncoder()
y = le.fit_transform(labels)

vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(texts).toarray()


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

y_cat = to_categorical(y)

model = Sequential()
model.add(Dense(128, input_shape=(X.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(y_cat.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y_cat, epochs=20, batch_size=8)


Epoch 1/20
347/347 [==============================] - 4s 6ms/step - loss: 1.6123 - accuracy: 0.1986
Epoch 2/20
347/347 [==============================] - 2s 7ms/step - loss: 1.5999 - accuracy: 0.2398
Epoch 3/20
347/347 [==============================] - 2s 7ms/step - loss: 1.5709 - accuracy: 0.2875
Epoch 4/20
347/347 [==============================] - 2s 7ms/step - loss: 1.5150 - accuracy: 0.3348
Epoch 5/20
347/347 [==============================] - 2s 6ms/step - loss: 1.4443 - accuracy: 0.3915
Epoch 6/20
347/347 [==============================] - 2s 7ms/step - loss: 1.3270 - accuracy: 0.4576
Epoch 7/20
347/347 [==============================] - 3s 8ms/step - loss: 1.2099 - accuracy: 0.5316
Epoch 8/20
347/347 [==============================] - 3s 8ms/step - loss: 1.1012 - accuracy: 0.5793
Epoch 9/20
347/347 [==============================] - 2s 7ms/step - loss: 0.9924 - accuracy: 0.6208
Epoch 10/20
347/347 [==============================] - 2s 7ms/step - loss: 0.8935 - accuracy: 0.6587

In [15]:
import numpy as np

def predict_intent(text):
    vec = vectorizer.transform([text]).toarray()
    pred = model.predict(vec)[0]
    intent = le.inverse_transform([np.argmax(pred)])[0]
    return intent

intent_to_response = df.groupby("Ticket Type")["Resolution"].apply(lambda x: x.sample(1).values[0]).to_dict()

def get_bot_response(msg):
    intent = predict_intent(msg)
    return intent_to_response.get(intent, "Sorry, I didn’t get that. Can you rephrase?")


In [16]:
from flask import Flask, request, render_template, jsonify

app = Flask(__name__)

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/get", methods=["POST"])
def chatbot_response():
    user_msg = request.form["msg"]
    bot_reply = get_bot_response(user_msg)
    return jsonify({"response": bot_reply})

if __name__ == "__main__":
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\roshi\AppData\Local\Programs\Python\Python311\Lib\site-packages\IPython\core\interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
<!DOCTYPE html>
<html>
<head>
    <title>Support Chatbot</title>
</head>
<body>
    <h2>Chat with Support Bot</h2>
    <form id="chat-form">
        <input id="msg" type="text" required>
        <button type="submit">Send</button>
    </form>
    <div id="chat-output"></div>

    <script>
        const form = document.getElementById("chat-form");
        form.onsubmit = async (e) => {
            e.preventDefault();
            const msg = document.getElementById("msg").value;
            const res = await fetch("/get", {
                method: "POST",
                headers: {"Content-Type": "application/x-www-form-urlencoded"},
                body: "msg=" + encodeURIComponent(msg)
            });
            const data = await res.json();
            document.getElementById("chat-output").innerText = "Bot: " + data.response;
        };
    </script>
</body>
</html>
